In [1]:
# If these things haven't been precomputed:
# - Run maps_data.ipynb to generate all_rooms.json
# - Run node_level_tla.ipynb to generate policy_id.ddmp

# Instructions for use (awful):
# Any order:
# - Run this notebook up to the while True cell
#   - Run the while True cell
# - In a terminal open in ./notebooks, run python3 server.py
#   - Open a browser (firefox) tab to localhost:8000/smmaps.html
# - Open snes9x-rr and load sm_clean.smc
#   - File -> Lua Scripting -> New Lua Script Window
#   - Browse and open ./notebooks/state_snes9x.lua
#   - Play through Ceres station or load the implicit save from after Ceres
#   - Walk to the left door of the Landing Site
# - Run the while True cell in this notebook

In [2]:
#TODO: move all_rooms.json, lines.json, and player_pos.json to output

In [3]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
import sys
sys.path.append("..")

In [5]:
import time
import json

In [6]:
advice_fn = """
area_offsets = {
    [0] = {[0] = 3, [1] = 10},
    [1] = {[0] = 0, [1] = 29},
    [2] = {[0] = 31, [1] = 49},
    [3] = {[0] = 37, [1] = 0},
    [4] = {[0] = 28, [1] = 29},
    [5] = {[0] = 0, [1] = 10},
    [6] = {[0] = 0, [1] = -10},
    [7] = {[0] = 0, [1] = 0}
}
local px_addr = 0x000AF6
local py_addr = 0x000AFA
local camx_addr = 0x000911
local camy_addr = 0x000915
local area_addr = 0x00079F
local mapx_addr = 0x0007A1
local mapy_addr = 0x0007A3
local camx = memory.readword(camx_addr)
local camy = memory.readword(camy_addr)
local px = memory.readword(px_addr)
local py = memory.readword(py_addr)
local screen_x = px - camx
local screen_y = py - camy
local xrad = memory.readbyte(0x7E0AFE)
local yrad = memory.readbyte(0x7E0B00)
local area = memory.readbyte(area_addr)
local mapx = memory.readbyte(mapx_addr)
local mapy = memory.readbyte(mapy_addr)
local px_global = 16 * area_offsets[area][0] + 16 * mapx + (px / 16)
local py_global = 16 * area_offsets[area][1] + 16 * mapy + (py / 16)
"""
advice2 = """
local go_x = {gox}
local go_y = {goy}
local to_x = go_x - px_global
local to_y = go_y - py_global
--print("hey")
--print(px_global, py_global)
--print(go_x, go_y)
--print(to_x, to_y)
gui.line(screen_x, screen_y, screen_x + (to_x * 16), screen_y + (to_y * 16), 0xffffffff)
local dist = math.sqrt((to_x * to_x + to_y * to_y))
gui.text(screen_x, screen_y, tostring(math.floor(dist)), 0xffffffff)
gui.text(0, 0, "Node Inference: {nit} ms", 0xffffffff)
gui.text(0, 10, "Path Inference: {pit} ms", 0xffffffff)
"""
def mk_advice(x, y, nit, pit):
    return advice_fn + advice2.format(gox=x, goy=y, nit=nit, pit=pit)

In [7]:
from bdds.node_bdds import MapsInfo
from rom_tools.rom_manager import RomManager

In [8]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")

In [9]:
map_info = MapsInfo("../encoding/dsl/rooms_for_alloy.txt", \
                   "../encoding/dsl/exits_for_alloy.txt",
                   "../output/ram_snes9x.bin",
                   rom)

Parsing ROM...
Building Dicts...
West_Ocean 0x793fe
	 Extra: ['West_Ocean_R4', 'West_Ocean_L']
	 Missing: []
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: []


In [10]:
#TODO: There will be an issue with West Ocean since the naming scheme isn't exact
# Keys in all_posns that don't have a corresponding node_id
set(map_info.all_posns.keys()) - set(map_info.node_ids.keys())

{'West_Ocean_L', 'West_Ocean_R4'}

In [11]:
# Node_ids that don't have a corresponding key in all_posns
set(map_info.node_ids.keys()) - set(map_info.all_posns.keys())

set()

In [12]:
map_info.node_ids["Landing_Site_Ship"]

0

In [13]:
map_info.all_posns["Landing_Site_L2"]

c(416.0,231.5)

In [14]:
map_info.all_posns["Landing_Site_R2"]

c(559.0,231.5)

In [15]:
from world_rando.coord import Coord

In [16]:
# Custom coords:
#TODO: need to do this for all_posns2 as well / instead
map_info.all_posns["Landing_Site_Ship"] = map_info.all_posns["Landing_Site_L2"] + Coord(71, 231.5)
map_info.all_posns["Landing_Site_End"] = map_info.all_posns["Landing_Site_L2"] + Coord(81, 231.5)

In [17]:
#with open("../output/policy_id.json", "r") as f:
#    var_levels = json.load(f)["level_of_var"]

In [18]:
#map_info.context.bdd.reorder(var_levels)
#map_info.context.bdd.configure(reordering=False)

In [19]:
#TODO: save var order and recall
# context1.bdd.reorder(context2.bdd.var_levels)
#policy_id = map_info.context.bdd.load("../output/policy_id.dddmp")[0]
policy_id = map_info.context.bdd.load("../output/node_policy_det.dddmp")[0]

In [20]:
policy_id

Function (DdNode) with var index: 29, ref count: 1, int repr: 94752479048451

In [21]:
policy_id.dag_size

234782

In [22]:
from data_types.item_set import ItemSet

In [23]:
# For passing player pos to the HTML interface
posd = {
    "X": None,
    "Y": None
}

In [24]:
from abstraction_validation.abstractify import abstractify_pos_global
from world_rando.coord import Coord

In [25]:
# The map on the website from bin0al uses a slightly different coordinate system
# These are the offsets for bin0al's map
map_area_offsets = {
0: Coord(3, 10),
1: Coord(0, 28),
2: Coord(31, 48),
3: Coord(37, 0),
4: Coord(28, 28),
5: Coord(0, 10),
#Out of bounds
6: Coord(0, -10),
7: Coord(0, 0),
};

In [26]:
def get_onmap_pos():
    ram_data = np.fromfile("../output/ram_snes9x.bin", dtype="int16")
    return abstractify_pos_global(ram_data, map_area_offsets)

In [27]:
from data_types.constraintgraph import Item, Boss

In [28]:
item_nodes = set([])
for r, room in map_info.rooms.items():
    for node in room.graph.name_node.keys():
        n = room.graph.name_node[node].data
        if isinstance(n, Item) or isinstance(n, Boss):
            item_nodes.add(node)

In [29]:
from itertools import pairwise

In [30]:
#TODO: Gross fix for the fact that map coords from bin0al don't match what I used...
#TODO: Croc speedway abstract nodes aren't matched correctly

In [31]:
from abstraction_validation.sm_paths import all_global_positions

In [32]:
%%capture
all_posns2 = all_global_positions(map_info.rooms, map_info.parsed_rom, map_area_offsets);

In [33]:
# Produce node list for html display
posn_info = {k: (v.x, v.y) for k,v in all_posns2.items()}
all_nodes = json.dumps(posn_info)
with open("all_nodes.json", "w") as f:
    f.write(all_nodes)

In [34]:
all_posns2["Morph_Ball_Room_EB"], map_info.all_posns["Morph_Ball_Room_EB"]

(c(423.5,576.0), c(423.5,592.0))

In [35]:
# Yellow: current segment
# Green: segment up to the next item
# Grey: rest of the lines
def get_path(edges):
    if len(edges) < 2:
        return None
    node_ids = [edges[0][1]] + [e[2] for e in edges]
    nodes = [map_info.node_ids_rev[i] for i in node_ids]
    return nodes

def path_helper(state_estimate, policy):
    return map_info.get_path_advice(state_estimate, policy)

def generate_lines(path):
    out = {
        "next_step": [],
        "next_item": [],
        "remaining_path": [],
    }
    # The "path" only makes sense if there's at least two locations
    if path is None or len(path) < 2:
        return out
    else:
        # Filter down to nodes that we know the location of
        #TODO: this has issues if there's an item node with an unknown location (e.g. a boss)
        path_locs = [(p, all_posns2[p]) for p in path if p in all_posns2]
        # Also a possibility if path is long enough but we don't know where enough of the locs are
        if len(path_locs) < 2:
            return out
        which_current = "next_step"
        which_next = "next_step"
        for (node, loc) in path_locs:
            which_current = which_next
            out[which_current].append(loc)
            # [a, b]
            # [b, c, ... e]
            # [e, ...]
            # Desired behavior: if the next node is an item node, next_item should be empty.
            if which_current == "next_step" and len(out["next_step"]) == 2:
                which_next = "next_item"
            if which_current == "next_item" and node in item_nodes:
                which_next = "remaining_path"
            if which_next != which_current:
                out[which_next].append(loc)
        return out

In [36]:
import json

In [37]:
import time

In [38]:
current_goal_node = "Landing_Site_End"

In [46]:
map_info.estimate_state()

In [47]:
current_estimate

('Landing_Site_L2', ISet([]))

In [ ]:
# X Only update position on door transition
# ? Abstraction is room-aware
# X Only update position when you're close to the node
# Abstractify can handle bosses
node_inference_time_ms = "No Clue"
path_inference_time_ms = "No Clue"

current_node = "Landing_Site_L2"
current_goal = "Landing_Site_L2"
map_info.current_goal_node = current_goal
current_itemset = ItemSet()
update = False
while True:
    # Check our current state estimate and track whether it changed
    current_estimate = map_info.estimate_state()
    if current_estimate is not None:
        temp_node, temp_itemset = current_estimate
        # New itemset info
        if temp_itemset != current_itemset:
            print(f"Player got new item: {temp_itemset}")
            update = True
        # New node info
        if temp_node != current_node:
                print(f"Player at new node: {temp_node}")
                update = True
    # Process and update the goal and track whether it changed
    with open("./goal.json", "r") as f:
        temp_goal = json.loads(f.read())["goal"]
    if temp_goal != map_info.current_goal_node and temp_goal in map_info.node_ids:
        print(f"Player has new goal: {temp_goal}")
        map_info.current_goal_node = temp_goal
        update = True
        
        #TODO: New goal
    if update:
        if current_estimate is None:
            current_estimate = (current_node, current_itemset)
            #print(current_estimate)
        update = False
        t0 = time.time()
        next_node = map_info.get_step_advice(current_estimate, policy_id)
        t1 = time.time()
        node_inference_time_ms = int((t1-t0) * 1000)
        if next_node is not None:
            current_node = temp_node
            current_itemset = temp_itemset
            print(f"Next Node: {next_node}")
            # Write out the new advice
            if next_node in map_info.all_posns:
                next_node_pos = map_info.all_posns[next_node]
                advice = mk_advice(next_node_pos.x, next_node_pos.y, node_inference_time_ms, path_inference_time_ms)
                f = open("../output/snes9x_bridge.lua", "w")
                f.write(advice)
                f.close()
            else:
                print(f"I don't know where {temp_node} is")
            # Write out the new line data
            edges = map_info.get_edge_advice2((current_node, current_itemset), policy_id)
            t0 = time.time()
            lines = generate_lines(get_path(edges))
            t1 = time.time()
            path_inference_time_ms = int((t1-t0) * 1000)
            lstr = json.dumps(lines)
            with open("lines.json", "w") as f:
                f.write(lstr)
        else:
            #print("Decided not to gaslight you into believing you were at {next_node}")
            pass
    # Update the position tracking file
    pos = get_onmap_pos()
    posd["X"] = int(pos.x)
    posd["Y"] = int(pos.y)
    dstr = json.dumps(posd)
    with open("player_pos.json", "w") as f:
        f.write(dstr)
    time.sleep(0.1)

#TODO: Add node names - Go from A to B + inference time to the UI
#TODO: Path file JSON? -> show on some kind of map
#TOOD: bio-64 and bio-128 / 1024?

# TODO: "skilled" advice using impossible items
# ~50% of within-room edges unguarded

Player at new node: Parlor_R3
Player has new goal: Landing_Site_End
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Player at new node: Parlor_R3
Pl

In [62]:
print(map_info.read_state().position)

c(419,231)


In [116]:
(current_node, current_itemset)

('Big_Pink_M2', ISet(['B', 'M', 'MB', 'CB', 'E']))

In [154]:
%%time
generate_lines((current_node, current_itemset), policy_id)

['Big_Pink_M2', 'Big_Pink_R1', 'PreSpore_Kihunters_L', 'PreSpore_Kihunters_T', 'Spore_Spawn_B', 'Spore_Spawn_Spore_Spawn', 'Spore_Spawn_R', 'Spore_Spawn_Supers_L1', 'Spore_Spawn_Supers_S', 'Spore_Spawn_Supers_S', 'Spore_Spawn_Supers_L2', 'Spore_Spawn_Farm_R', 'Spore_Spawn_Farm_L', 'Big_Pink_R3', 'Big_Pink_R4', 'Green_Hill_Zone_L', 'Green_Hill_Zone_R2', 'Noob_Bridge_L', 'Noob_Bridge_R', 'Red_Tower_L1', 'Red_Tower_R2', 'Bat_Room_L', 'Bat_Room_R', 'Spazer_Troll_L', 'Spazer_Troll_R1', 'Spazer_L', 'Spazer_Spazer', 'Spazer_Spazer', 'Spazer_L', 'Spazer_Troll_R1', 'Spazer_Troll_R2', 'West_Tunnel_L', 'West_Tunnel_R', 'Glass_Tunnel_L', 'Glass_Tunnel_R1', 'East_Tunnel_L', 'East_Tunnel_R2', 'Warehouse_Entrance_L', 'Warehouse_Entrance_R', 'Warehouse_Zeelas_L1', 'Warehouse_Zeelas_T', 'Warehouse_Kihunters_B', 'Warehouse_Kihunters_R2', 'Baby_Kraid_L', 'Baby_Kraid_R', 'Kraid_Eye_Door_L', 'Kraid_Eye_Door_R2', 'Kraid_L', 'Kraid_Kraid', 'Kraid_Kraid', 'Kraid_R', 'Varia_Suit_L', 'Varia_Suit_V', 'Varia_Suit

{'next_step': [(c(274,631), c(303.0,535.5))],
 'next_item': [(c(303.0,535.5), c(304.0,535.5)),
  (c(304.0,535.5), c(359.5,528.0)),
  (c(359.5,528.0), c(359.5,527.0)),
  (c(359.5,527.0), c(367.0,487.5)),
  (c(367.0,487.5), c(368.0,487.5)),
  (c(368.0,487.5), c(394,615))],
 'remaining_path': [(c(394,615), c(394,615)),
  (c(394,615), c(368.0,615.5)),
  (c(368.0,615.5), c(367.0,615.5)),
  (c(367.0,615.5), c(320.0,615.5)),
  (c(320.0,615.5), c(319.0,615.5)),
  (c(319.0,615.5), c(303.0,631.5)),
  (c(303.0,631.5), c(304.0,631.5)),
  (c(304.0,631.5), c(431.0,679.5)),
  (c(431.0,679.5), c(432.0,679.5)),
  (c(432.0,679.5), c(527.0,679.5)),
  (c(527.0,679.5), c(528.0,679.5)),
  (c(528.0,679.5), c(543.0,759.5)),
  (c(543.0,759.5), c(544.0,759.5)),
  (c(544.0,759.5), c(575.0,759.5)),
  (c(575.0,759.5), c(576.0,759.5)),
  (c(576.0,759.5), c(607.0,743.5)),
  (c(607.0,743.5), c(608.0,743.5)),
  (c(608.0,743.5), c(619,745)),
  (c(619,745), c(619,745)),
  (c(619,745), c(608.0,743.5)),
  (c(608.0,743.5),